In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
import sys
import os

In [3]:
from selenium import webdriver

In [6]:
from collections import defaultdict

In [7]:
def flushprint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %www)
    sys.stdout.flush()

In [8]:
food_score=pd.read_excel('beijing_score.xls')

In [11]:
review_content=[]

In [ ]:
for index in range(0,6631):
    review_content_=[]
    url='https://www.tripadvisor.cn'+food_score.iloc[index]['food-link']
    food_name=food_score.iloc[index]['food-name']
    food_link=food_score.iloc[index]['food-link']
    score=food_score.iloc[index]['food-score']
    review_num=food_score.iloc[index]['review-num']
    review=[food_name,food_link,score,review_num]
    try:
        tt = random.random()
        flushprint(str(index)+':'+str(url)+':sleeping'+str(tt)+'seconds')
        time.sleep(tt)
        
        html=requests.get(url)
        html.encoding='utf-8'
        content=html.content
        soup=BeautifulSoup(content,'html.parser')
        try:
            if soup.select('#REVIEWS #taplc_detail_filters_rr_resp_0 .is-shown-at-tablet .ui_columns .prw_filters_detail_language .content .is-shown-at-tablet .taLnk'):
                browser = webdriver.Chrome()
                browser.get(url)
                browser.find_element_by_xpath('//*[@id="taplc_detail_filters_rr_resp_0"]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[5]').click()

                source=browser.page_source
                soup2=BeautifulSoup(source, 'html.parser')
                for item in soup2.select('.prw_filters_detail_language .body_text .more-options .item .label'):
                    review_content_.append(item.text)
                review.append(review_content_)
                review_content.append(review)
                browser.close()
            else:
                for item in soup.select('#REVIEWS #taplc_detail_filters_rr_resp_0 .is-shown-at-tablet .ui_columns .prw_filters_detail_language .content .choices .item .label'):
                    review_content_.append(item.text)
                review.append(review_content_)
                review_content.append(review)
        except Exception as w:
            print(index,url,w)
            review_content_='NaN'
            review.append(review_content_)
            review_content.append(review)
    except Exception as e:
            print(index,url,e)
            continue

In [14]:
df=pd.DataFrame(review_content,columns=['food-name','food-link','score','review-num','review-content'])

In [15]:
df.to_excel('beijing_review.xls')